# Open Notebook Dependency Fix Documentation

This notebook documents the steps to resolve dependency issues with the Open Notebook CLI in Docker.

## Identified Issues

Based on the error logs, we've identified several dependency issues:

1. Missing packages: streamlit, podcastfy, pyyaml (import error)
2. Database connection issue: missing `sblpy` package (for SurrealDB connectivity)

The primary error is:
```
ModuleNotFoundError: No module named 'sblpy'
```

This is occurring when trying to run the Open Notebook CLI.

## Solution Approach

We'll create a comprehensive fix script that installs all required dependencies, including the newly identified `sblpy` package.

In [ ]:
# Complete Fix Script (complete_fix.py)
#!/usr/bin/env python3

import subprocess
import sys
import os
from datetime import datetime

def log(message, level="INFO"):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}] [{level}] {message}")

def run_command(command, description=None, check=True):
    if description:
        log(description)
    
    try:
        result = subprocess.run(
            command,
            check=check,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        print(result.stdout)
        if result.stderr and not result.stderr.isspace():
            print(f"STDERR: {result.stderr}")
        return result
    except subprocess.CalledProcessError as e:
        log(f"Error executing command: {command}", "ERROR")
        log(f"Return code: {e.returncode}", "ERROR")
        if e.stderr:
            log(f"STDERR: {e.stderr}", "ERROR")
        if not check:
            return e
        else:
            return None

def verify_import(package_name, container="assistant-notebook-1"):
    log(f"Checking if {package_name} is properly installed...")
    result = run_command(
        ["docker", "exec", container, "python", "-c", 
         f"import {package_name}; print('{package_name} successfully imported!')"],
        check=False
    )
    
    if result.returncode == 0:
        return True
    else:
        log(f"ERROR: Failed to import {package_name}!", "ERROR")
        return False

def install_packages():
    # Define all packages to install
    packages = [
        "pydantic",
        "langchain",
        "langgraph",
        "streamlit",
        "podcastfy",
        "typing_extensions",
        "humanize",
        "pyyaml",
        "sblpy"  # Add the missing SurrealDB library
    ]
    
    for package in packages:
        log(f"Installing {package}...")
        run_command(
            ["docker", "exec", "assistant-notebook-1", "pip", "install", package],
            check=False
        )
    
    # Verify all packages
    all_installed = True
    for package in packages:
        if not verify_import(package):
            all_installed = False
    
    if all_installed:
        log("All dependencies successfully installed!")
    else:
        log("WARNING: Some dependencies could not be imported.", "WARNING")
        log("The CLI may not work correctly.", "WARNING")

def copy_cli_script():
    log("\n====== Copying CLI Script to Container ======")
    run_command(
        ["docker", "cp", "open_notebook_cli.py", "assistant-notebook-1:/app/"],
        "Copying CLI script to container..."
    )

def run_test_command():
    log("\n====== Running Test Command ======")
    log("Running 'list-notebooks' command...")
    result = run_command(
        ["docker", "exec", "assistant-notebook-1", "python", "/app/open_notebook_cli.py", "list-notebooks"],
        check=False
    )
    
    if result.returncode != 0:
        log("WARNING: Test command failed. The CLI might not be working correctly.", "WARNING")
        return False
    return True

def main():
    log("====== Starting Dependency Fix ======")
    
    # Install all required packages
    install_packages()
    
    # Copy CLI script to container
    copy_cli_script()
    
    # Run test command
    success = run_test_command()
    
    if success:
        log("\n===== Fix Complete =====")
        log("All dependencies are now installed and the CLI is working correctly.")
    else:
        log("\n===== Fix Incomplete =====")
        log("Some issues remain. Please check the error messages above.")

if __name__ == "__main__":
    main()

## Quick Fix Shell Script

Alternatively, you can use this quick shell script to fix the dependencies:

In [ ]:
#!/bin/bash

# Quick fix script for Open Notebook dependencies
echo "Installing missing dependencies..."

# Install the missing sblpy package
docker exec assistant-notebook-1 pip install sblpy

# Verify the sblpy package is installed
docker exec assistant-notebook-1 python -c "import sblpy; print('sblpy successfully imported!')"

# Also ensure other potentially missing packages are installed
docker exec assistant-notebook-1 pip install streamlit podcastfy pyyaml

echo "Testing CLI..."
docker exec assistant-notebook-1 python /app/open_notebook_cli.py list-notebooks

echo "Fix completed."

## Testing the Fix

After applying either fix method, test the CLI with:

In [ ]:
./on_run.sh list-notebooks

## Understanding the Error

The primary issue was a missing dependency `sblpy` which is required for SurrealDB connectivity. This package is used by the Open Notebook application to connect to its database.

The dependency chain is:
```
open_notebook_cli.py -> open_notebook.domain.notebook -> open_notebook.database.repository -> sblpy.connection
```

When this dependency is missing, the CLI cannot initialize properly.

## Alternative Solution: Creating a Standalone Script

If you prefer not to modify the container, you can create a standalone script that does not depend on the database connection.

## Advantages of the Enhanced Wrapper

Compared to the regular wrapper, the enhanced version:

1. Automatically handles missing dependencies
2. Provides clearer error messages
3. Adds a convenience command for installing all common dependencies
4. Falls back gracefully when installation methods fail

This makes it more robust for environments where the container might not have all required dependencies pre-installed.